In [1]:
!pip install rpy2==3.5.1

In [2]:
%load_ext rpy2.ipython

Unable to determine R home: [WinError 2] The system cannot find the file specified
C:\Anaconda\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
%%R
# data preprocessing
library(tidyverse)
library(lubridate)
# data exploration
library(summarytools) # for user-friendly html summaries of data
library(ggmap) # for plotting data on a map
# for meta-ml
library(tidymodels)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.3.6      v purrr   0.3.5 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.1      v stringr 1.4.1 
v readr   2.1.3      v forcats 0.5.2 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


R[write to console]: Loading required package: timechange

R[write to console]: 
Attaching package: 'lubridate'


R[write to console]: The following objects are masked from 'package:base':

    date, intersect, setdiff, union


R[write to console]: 
Attaching package: 'summarytools'


R[write to console]: The following object is masked from 'package:tibble':

    view




i Google's Terms of Service: <]8;;https://mapsplatform.google.comhttps://mapsplatform.google.com]8;;>
i Please cite ggmap if you use it! Use `citation("ggmap")` for details.


R[write to console]: -- Attaching packages ------------------------------------------------------------------------------ tidymodels 1.0.0 --

R[write to console]: v broom        1.0.1     v rsample      1.1.1
v dials        1.1.0     v tune         1.0.1
v infer        1.0.4     v workflows    1.1.2
v modeldata    1.0.1     v workflowsets 1.0.0
v parsnip      1.0.3     v yardstick    1.1.0
v recipes      1.0.3     

R[write to console]: -- Conflicts --------------------------------------------------------------------------------- tidymodels_conflicts() --
x scales::discard()    masks purrr::discard()
x dplyr::filter()      masks stats::filter()
x recipes::fixed()     masks stringr::fixed()
x dplyr::lag()         masks stats::lag()
x yardstick::spec()    masks readr::spec()
x recipes::step()      masks stats::step()
x summarytools::view() masks tibble::view()
* Learn how to get started at https://www.tidymodels.org/start/



In [4]:
%%R
# let's set some global options
options(dplyr.width = Inf) # show all columns when printing to console
theme_set(theme_minimal()) # select a lightweight ggplot theme for cleaner plotting
set.seed(2023) # in the AC, you'll be required to set a fixed random seed to make your work reproducible

In [5]:
%%R
# Start reading data
classification <- read_csv( 'classification.csv' )

customers <- read_csv(
  'customers.csv',
  col_types = cols(
    Item_Position = col_double() 
  )
)#change from character like "005230" to numeric number "5230"

sales_orders <- read_csv( 'sales_orders.csv')

sales_orders_header <- read_csv('sales_orders_header.csv' ) %>% 
  mutate(
    Release_Date = as_date(Release_Date) %>% as.character(),
    Creation_Date = as_date(Creation_Date) %>% as.character()
  )
#which(is.na(sales_orders_header$Release_Date)) #look if as_data is not run, exact time00:00:00 is added.
# 150048 header$Sales_Order, compared to 150064 sales_order$Sales_Order

business_units <- read_csv('business_units.csv') %>% 
  select( -YHKOKRS ) 
# YHKOKRS, all elements are 5180, useless

service_map <- read_csv('service_map.csv') 
#MATKL_service 29 distict values, compared to sales_orders$Material_Class with 329 distict values

R[write to console]: 
R[write to console]: indexing classification.csv [===============================] 1.18GB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 2561 Columns: 3
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (1): Customer_ID
dbl (2): Reseller, Test_set_id

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing customers.csv [===---------------------------------] 2.15GB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing customers.csv [==================================] 545.89MB/s, eta:  0s
                                                                                                                   
R[write to console]: 
R[write to console]: 
R[write to console]: indexing sales_orders.csv [---------------------------------] 2.15GB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [=============------------------] 552.12MB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [===============----------------] 555.49MB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [==================-------------] 553.82MB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [====================-----------] 556.49MB/s, eta

Rows: 1248889 Columns: 7
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (3): Sales_Order, Material_Code, Cost_Center
dbl (4): Item_Position, Num_Items, Material_Class, Net_Value

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing sales_orders_header.csv [==------------------------] 2.15GB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders_header.csv [========================] 515.43MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 150058 Columns: 8
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (6): Sales_Organization, Sales_Order, Creator, Document_Type, Release_D...
dbl  (1): Net_Value
dttm (1): Creation_Date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing business_units.csv [=============================] 648.19MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 126 Columns: 3
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): Cost_Center, Business_Unit
dbl (1): YHKOKRS

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing service_map.csv [=================================] 11.97MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 29 Columns: 1
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
dbl (1): MATKL_service

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
%%R
#Start joining tibble data
cla_cus <- left_join(classification, customers, by = c('Customer_ID' = 'Customer_ID'))

#join sales order
cla_cus_sal_inner <- inner_join(cla_cus, sales_orders, by = c('Sales_Order' = 'Sales_Order','Item_Position' = 'Item_Position'))

sales_orders_rest <- anti_join(sales_orders, cla_cus, by = c('Sales_Order' = 'Sales_Order','Item_Position' = 'Item_Position')  )
sales_orders_rest <- sales_orders_rest %>% mutate(Item_Position = 0 )
cla_cus_sal_rest <- inner_join(cla_cus, sales_orders_rest, by = c('Sales_Order' = 'Sales_Order','Item_Position' = 'Item_Position'))

cla_cus_sal <- bind_rows(cla_cus_sal_inner, cla_cus_sal_rest)
#cla_cus_sal %>% dfSummary %>% view  #cla_cus_sal$Test_set_id has 512 distinct values, which is same as classification.csv

#join sales order header
cla_cus_sal_head <- left_join(cla_cus_sal, sales_orders_header, by = c('Sales_Order' = 'Sales_Order'))

#join business unit
cla_cus_sal_head_busi <- left_join(cla_cus_sal_head, business_units, by = c('Cost_Center' = 'Cost_Center'))
all_merged <- cla_cus_sal_head_busi

#join service maps
all_merged = all_merged %>%
  mutate(
    MATKL = case_when(
      Material_Class %in% service_map$MATKL_service ~ 1,
      TRUE ~ 0 # TRUE is equivalent to an “else” statement.
    )
  )
#all_merged %>% dfSummary %>% view

In [7]:
%%R
print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(all_merged)) { # for-loop over columns
  print(paste0(names(all_merged)[i], ": ", sum(is.na(all_merged[, i])), "/", nrow(all_merged)))
}
print("[INFO] Glimpse:")
glimpse(all_merged)

#join and merge is finished above
#####################################################################################
df <- all_merged

[1] "[INFO] Number of NA values in each column:"
[1] "Customer_ID: 0/1409894"
[1] "Reseller: 305813/1409894"
[1] "Test_set_id: 1104081/1409894"
[1] "Sales_Order: 0/1409894"
[1] "Item_Position: 0/1409894"
[1] "Type: 0/1409894"
[1] "Num_Items: 0/1409894"
[1] "Material_Code: 0/1409894"
[1] "Material_Class: 2/1409894"
[1] "Cost_Center: 0/1409894"
[1] "Net_Value.x: 0/1409894"
[1] "Sales_Organization: 43/1409894"
[1] "Creation_Date: 43/1409894"
[1] "Creator: 43/1409894"
[1] "Document_Type: 43/1409894"
[1] "Release_Date: 3683/1409894"
[1] "Delivery: 43/1409894"
[1] "Net_Value.y: 43/1409894"
[1] "Business_Unit: 0/1409894"
[1] "MATKL: 0/1409894"
[1] "[INFO] Glimpse:"
Rows: 1,409,894
Columns: 20
$ Customer_ID        <chr> "0x519277E9B1C951A10E3C688949F69A00C623ADC26AADD09B~
$ Reseller           <dbl> 0, 0, 0, 0, 0, NA, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,~
$ Test_set_id        <dbl> NA, NA, NA, NA, NA, 137, NA, NA, NA, NA, NA, NA, NA~
$ Sales_Order        <chr> "0xE52DCC1B2EF98A9D36D30F9C91514285612

In [8]:
%%R
###Handling binary features
#  "Type": Replace string values "SOP,STP" with binary values
df = df %>%
  mutate(
    Type_SOTP = case_when(
      Type %in% c("SOP") ~ 1,
      Type %in% c("STP") ~ 0,
    )
  )
df = select (df,-c(Type))

#  "Sales_Organization": Replace string values "A,B " with binary values
df = df %>%
  mutate(
    Organization = case_when(
      Sales_Organization %in% c("A") ~ 1,
      Sales_Organization %in% c("B") ~ 0,
    )
  )
df = select (df,-c(Sales_Organization))

#  "Creator": "0x2500D9>69.7%"
df = df %>%
  mutate(
    Creator_Binary = case_when(
      Creator == "0x2500D9E578134DB667D820EA3100F880DAF13E408E6761CEE69DBFC3EAF833B4" ~ 1,
      TRUE ~ 0
    )
  )

In [9]:
%%R
###Handling date
#  "Creation_Date" and "Release_Date"
# Compare them with each other first
df = df %>%
  mutate(crea_relea = case_when(
    Creation_Date==Release_Date ~ 1,
    TRUE ~ 0
  ))
#df$crea_relea %>% dfSummary %>% view
df <- df %>% select(-crea_relea)
# Creation_Date 
#year
df = df %>%
  mutate(Creation_Year = case_when(
    !is.na(Creation_Date) ~ Creation_Date %>%
      substr(nchar(Creation_Date) - 9, nchar(Creation_Date)-6 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Creation_Year = (Creation_Year-2020)* 365 )
#month
df = df %>%
  mutate(Creation_Month = case_when(
    !is.na(Creation_Date) ~ Creation_Date %>%
      substr(nchar(Creation_Date) - 4, nchar(Creation_Date)-3 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Creation_Month = ( Creation_Month-1)* 30 )
#day
df = df %>%
  mutate(Creation_Day = case_when(
    !is.na(Creation_Date) ~ Creation_Date %>%
      substr(nchar(Creation_Date) - 1, nchar(Creation_Date) ) %>%
      as.numeric()
  ))
df = df %>% mutate(Creation_Date = Creation_Date )
#accumulate date
df <- df %>% mutate(Creation_Date_New = Creation_Year+Creation_Month+Creation_Day  )
df <- df %>% select(-Creation_Year, -Creation_Month, -Creation_Day)

#Release_Date
#year
df = df %>%
  mutate(Release_Year = case_when(
    !is.na(Release_Date) ~ Release_Date %>%
      substr(nchar(Release_Date) - 9, nchar(Release_Date)-6 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Release_Year = (Release_Year-2020)* 365 )
#month
df = df %>%
  mutate(Release_Month = case_when(
    !is.na(Release_Date) ~ Release_Date %>%
      substr(nchar(Release_Date) - 4, nchar(Release_Date)-3 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Release_Month = ( Release_Month-1)* 30 )
#day
df = df %>%
  mutate(Release_Day = case_when(
    !is.na(Release_Date) ~ Release_Date %>%
      substr(nchar(Release_Date) - 1, nchar(Release_Date) ) %>%
      as.numeric()
  ))
df = df %>% mutate(Release_Date = Release_Date )
#accumulate date
df <- df %>% mutate(Release_Date_New = Release_Year+Release_Month+Release_Day  )
df <- df %>% select(-Release_Year, -Release_Month, -Release_Day)

#Difference between release and creation data
df <- df %>% mutate(Diff_Date = Release_Date_New-Creation_Date_New  )

#df %>% dfSummary %>% view

In [10]:
%%R
##############################################################################
### Missing data handling 
#Material_Class  2
df[which(is.na(df$	Material_Class)),] 
#which(is.na(df$	Material_Class))
#df[866465:866474,]
#df[866531:866539,]
#not in Test_set, delete
df <- df[-which(is.na(df$	Material_Class)),]


#sales order header: 43/1409894
missing1 <- df[which(is.na(df$Creation_Date_New)),] 
#missing1 %>% dfSummary %>% view
#Diff_Date: 3683/1409894
missing2 <- df[which(is.na(df$Diff_Date)),] 
#missing2 %>% dfSummary %>% view
#include test data, so they cannot be deleted

ddf <- df

ddf[which(is.na(ddf$Document_Type)),]$Document_Type <- "Order"  #"Order" accounts for 99%, so...
ddf[which(is.na(ddf$Delivery)),]$Delivery <- "Not relevant"   #"Not relevant" accounts for 93.3%, so...
ddf[which(is.na(ddf$Net_Value.y)),]$Net_Value.y <- 615456    #  615455.6 is average of Net_Value.y, so...
ddf[which(is.na(ddf$Organization)),]$Organization <- 0  #Organization: 0 accounts for 67.7% and 1 is 32.3%, so..
ddf[which(is.na(ddf$Creation_Date_New)),]$Creation_Date_New <- 533 #533.3 is average of Creation_Date_New, so...
ddf[which(is.na(ddf$Release_Date_New)),]$Release_Date_New <- 542#542 is average of Release_Date_New, so...
ddf[which(is.na(ddf$Diff_Date)),]$Diff_Date <- 8     #8.2 is average of Diff_Date, so...

#ddf %>% dfSummary %>% view

#Cost_Center could be further used
#Document_Type, Order accounts for 99%, could be further used.
#Both Net_Values are not handled. Maybe log function


dfdf <- ddf %>% select(-c(Customer_ID, Sales_Order, Material_Code, Creation_Date ,Release_Date))
#dfdf %>% dfSummary %>% view

In [11]:
%%R
#Split Data into Labeled and Unlabeled (Test) Sets
labeled_data = dfdf[is.na(dfdf$Test_set_id),]
test_data = dfdf[!is.na(dfdf$Test_set_id),]

labeled_data %>% nrow()
test_data %>% nrow()


write_csv(x = dfdf, file = "train_test_mixed.csv")
write_csv(x = labeled_data, file = "labeled_data.csv")
write_csv(x = test_data, file = "test_data.csv")


R[write to console]: 
R[write to console]: wrote 2.88MB in  0s, 2.15GB/s
R[write to console]: 
R[write to console]: wrote 95.02MB in  0s, 462.31MB/s
R[write to console]: 
R[write to console]: wrote 97.89MB in  0s, 456.25MB/s
R[write to console]: 
R[write to console]: wrote 100.78MB in  0s, 448.95MB/s
R[write to console]: 
R[write to console]: wrote 103.65MB in  0s, 441.90MB/s
R[write to console]: 
R[write to console]: wrote 106.53MB in  0s, 437.76MB/s
R[write to console]: 
R[write to console]: wrote 109.40MB in  0s, 430.24MB/s
R[write to console]: 
R[write to console]: wrote 112.27MB in  0s, 425.90MB/s
R[write to console]: 
R[write to console]: wrote 115.16MB in  0s, 420.20MB/s
R[write to console]: 
R[write to console]: wrote 118.05MB in  0s, 408.76MB/s
R[write to console]: 
R[write to console]: wrote 120.93MB in  0s, 402.19MB/s
R[write to console]: 
R[write to console]: wrote 123.82MB in  0s, 396.31MB/s
R[write to console]: 
R[write to console]: wrote 126.70MB in  0s, 392.55MB/s
R[wri

R[write to console]: 
R[write to console]: wrote 90.08MB in  0s, 433.12MB/s
R[write to console]: 
R[write to console]: wrote 92.33MB in  0s, 424.56MB/s
R[write to console]: 
R[write to console]: wrote 94.58MB in  0s, 424.21MB/s
R[write to console]: 
R[write to console]: wrote 96.82MB in  0s, 417.06MB/s
R[write to console]: 
R[write to console]: wrote 99.07MB in  0s, 413.73MB/s
R[write to console]: 
R[write to console]: wrote 101.33MB in  0s, 413.59MB/s
R[write to console]: 
R[write to console]: wrote 103.59MB in  0s, 405.54MB/s
R[write to console]: 
R[write to console]: wrote 105.85MB in  0s, 404.49MB/s
R[write to console]: 
R[write to console]: wrote 108.10MB in  0s, 407.22MB/s
R[write to console]: 
R[write to console]: wrote 110.35MB in  0s, 402.01MB/s
R[write to console]: 
R[write to console]: wrote 112.60MB in  0s, 398.74MB/s
R[write to console]: 
R[write to console]: wrote 114.86MB in  0s, 398.40MB/s
R[write to console]: 
R[write to console]: wrote 117.10MB in  0s, 391.12MB/s
R[wr

R[write to console]: 
R[write to console]: wrote 51.80MB in  0s, 216.14MB/s
R[write to console]: 
R[write to console]: wrote 52.43MB in  0s, 214.46MB/s
R[write to console]: 
R[write to console]: wrote 53.06MB in  0s, 213.72MB/s
R[write to console]: 
R[write to console]: wrote 53.69MB in  0s, 213.25MB/s
R[write to console]: 
R[write to console]: wrote 54.32MB in  0s, 211.12MB/s
R[write to console]: 
R[write to console]: wrote 54.95MB in  0s, 209.66MB/s
R[write to console]: 
R[write to console]: wrote 55.59MB in  0s, 207.80MB/s
R[write to console]: 
R[write to console]: wrote 56.22MB in  0s, 206.09MB/s
R[write to console]: 
R[write to console]: wrote 56.85MB in  0s, 204.99MB/s
R[write to console]: 
R[write to console]: wrote 57.48MB in  0s, 203.64MB/s
R[write to console]: 
R[write to console]: wrote 58.11MB in  0s, 202.64MB/s
R[write to console]: 
R[write to console]: wrote 58.74MB in  0s, 201.51MB/s
R[write to console]: 
R[write to console]: wrote 59.37MB in  0s, 200.36MB/s
R[write to c

In [12]:
#Python Classifier

In [13]:
!pip install pandas==1.5.2
!pip install scikit-learn==1.2.0
!pip install numpy==1.23.0

  Using cached numpy-1.22.4-cp38-cp38-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Anaconda\\Lib\\site-packages\\~-7py\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached numpy-1.23.0-cp38-cp38-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.0 which is incompatible.


In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from typing import List

C:\Anaconda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [15]:
#test dataset

In [16]:
read_data = pd.read_csv('train_test_mixed.csv') 
tabel = read_data

In [17]:
'''
#customers = labeled_data['Customer_ID'].unique()
customers = read_data.groupby(['Customer_ID']).count().reset_index().sort_values(by=['Reseller'],ascending=False)
customers_300 = customers[0:1000]
customers_300.tail(3)
'''

"\n#customers = labeled_data['Customer_ID'].unique()\ncustomers = read_data.groupby(['Customer_ID']).count().reset_index().sort_values(by=['Reseller'],ascending=False)\ncustomers_300 = customers[0:1000]\ncustomers_300.tail(3)\n"

In [18]:
'''
Customer_ID_300 = customers_300['Customer_ID'].to_numpy()
#Customer_ID_300
tabel = read_data
tabel.loc[~tabel['Customer_ID'].isin(Customer_ID_300),['Customer_ID']] = 'not_specified'
#labeled[~labeled['Customer_ID'].isin(Customer_ID_300)]
tabel
'''

"\nCustomer_ID_300 = customers_300['Customer_ID'].to_numpy()\n#Customer_ID_300\ntabel = read_data\ntabel.loc[~tabel['Customer_ID'].isin(Customer_ID_300),['Customer_ID']] = 'not_specified'\n#labeled[~labeled['Customer_ID'].isin(Customer_ID_300)]\ntabel\n"

In [19]:
df = pd.get_dummies(tabel)
df.head()

,Reseller,Test_set_id,Item_Position,Num_Items,Material_Class,Net_Value.x,Net_Value.y,MATKL,Type_SOTP,Organization,...,Document_Type_Order,Document_Type_Order w/o charge,Document_Type_Returns,Delivery_Completely processed,Delivery_Not relevant,Delivery_Not yet processed,Delivery_Partially processed,Business_Unit_BU_A,Business_Unit_BU_B,Business_Unit_BU_C
0,0.0,NaN,1000,12.0,81,602656,597357,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1,0.0,NaN,2000,1.0,343,1127711,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
2,0.0,NaN,3000,1.0,343,1129411,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
3,0.0,NaN,1000,12.0,343,987825,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
4,0.0,NaN,1040,1.0,68,2963102,2963102,0,0,1,...,1,0,0,1,0,0,0,1,0,0


In [20]:
train_data = df[df[['Test_set_id']].isnull().T.any()]
#test_data = df[!is.na(df$Test_set_id),]
train_data.shape

(1104079, 195)

In [21]:
test_data = df[df[['Test_set_id']].notnull().T.any()]
test_data.shape

(305813, 195)

In [22]:
test_data

,Reseller,Test_set_id,Item_Position,Num_Items,Material_Class,Net_Value.x,Net_Value.y,MATKL,Type_SOTP,Organization,...,Document_Type_Order,Document_Type_Order w/o charge,Document_Type_Returns,Delivery_Completely processed,Delivery_Not relevant,Delivery_Not yet processed,Delivery_Partially processed,Business_Unit_BU_A,Business_Unit_BU_B,Business_Unit_BU_C
5,NaN,137.0,4030,1.0,68,2963102,93915,0,0,1,...,1,0,0,1,0,0,0,1,0,0
23,NaN,408.0,1000,1.0,221,1168838,2963102,0,0,1,...,1,0,0,1,0,0,0,1,0,0
24,NaN,408.0,2000,3.0,221,1086675,2963102,0,0,1,...,1,0,0,1,0,0,0,1,0,0
26,NaN,641.0,2000,1.0,259,1029029,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
27,NaN,641.0,286003,0.0,227,829516,829516,1,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409865,NaN,2560.0,0,1.0,222,890541,890541,0,1,1,...,1,0,0,0,1,0,0,1,0,0
1409866,NaN,2560.0,0,2.0,100,74342,690628,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1409867,NaN,2560.0,0,1.0,100,779745,690628,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1409868,NaN,2560.0,0,1.0,100,793431,763667,0,1,0,...,1,0,0,0,1,0,0,0,0,1


In [23]:
labels = train_data['Reseller']
Test_set_id = test_data['Test_set_id']

In [24]:
training = train_data
testing = test_data
training = train_data.drop(['Reseller', 'Test_set_id','Creator_Binary'], axis=1)
testing = test_data.drop(['Reseller', 'Test_set_id', 'Creator_Binary'], axis=1)
#drop_col = ['Customer_ID', 'Reseller', 'Test_set_id', 'Creation_Date_New','Release_Date_New']

In [25]:
# Instantiate model 
rf = RandomForestClassifier(random_state=0) #parametres to be tuned

# Train the model on training data
rf.fit(training, labels)

RandomForestClassifier(random_state=0)

In [26]:
test_preds = rf.predict(testing)
test_preds.mean()

0.2059788171202663

In [27]:
np.isnan(test_preds).any()

False

In [28]:
#idx = pd.Index([test_preds ])
summary = pd.Series(test_preds)
#summary.describe()
summary.isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
305808    False
305809    False
305810    False
305811    False
305812    False
Length: 305813, dtype: bool

In [29]:
Test_set_id = Test_set_id.to_numpy().astype(int)

In [30]:
#ID_PRED = pd.Series(range(len(Test_set_id)))
ID_PRED = pd.DataFrame()
#ID_PRED['']=pd.Series(np.arange(len(Test_set_id)))
ID_PRED['Test_set_id']=pd.Series(Test_set_id)
ID_PRED['test_preds']=pd.Series(test_preds)
ID_PRED.tail(5)

,Test_set_id,test_preds
305808,2560,0.0
305809,2560,0.0
305810,2560,1.0
305811,2560,1.0
305812,2560,0.0


In [31]:
ID_PRED[ID_PRED[['test_preds']].isnull().T.any()]

,Test_set_id,test_preds


In [44]:
#ID_PRED.value_counts()
ID = ID_PRED['Test_set_id'].unique()
ID = ID.astype(int)

In [45]:
#ID_PRED[ID_PRED['Test_set_id'] == ID[1]]
#ID_PRED[ID_PRED['Test_set_id'] == ID[5]].mean()
#ID_PRED.iloc[1,:].shape

In [51]:
submit = {}
k = 0
for testid in list(ID):
    submit[k,] = ( ID_PRED[ID_PRED['Test_set_id'] == testid].mean() )
    k = k+1    
submit = pd.DataFrame(submit)
submit = submit.T
submit.head(3)

,Test_set_id,test_preds
0,137.0,0.135802
1,408.0,0.083333
2,641.0,0.699751


In [52]:
#submit['test_preds'] = submit['test_preds'].round().astype(int)
#submit[submit['test_preds']<=0.5]['test_preds'] = 0
submit.loc[submit['test_preds']<=0.5, ['test_preds'] ] = 0
submit.loc[submit['test_preds']>0.5, ['test_preds'] ] = 1
submit = submit.sort_values('Test_set_id')
submit['test_preds'] = submit['test_preds'].astype(int)
submit.head(3)

,Test_set_id,test_preds
15,11.0,0
16,14.0,0
17,17.0,0


In [53]:
 pub_K8PzhiD = pd.read_csv('pub_K8PzhiD.csv') 

In [54]:
#pub_K8PzhiD.set_index('id').join(submit, lsuffix='_caller', rsuffix='_other')
final = pub_K8PzhiD.join(submit.set_index('Test_set_id'), on = 'id')
#final.describe()
final = final.drop(['prediction'], axis=1)
final = final.rename(columns = {"id": "id","test_preds":"prediction"} )
final.describe()
#final

,id,prediction
count,512.000000,512.000000
mean,1278.757812,0.060547
std,727.019291,0.238731
min,11.000000,0.000000
25%,638.750000,0.000000
50%,1330.000000,0.000000
75%,1892.500000,0.000000
max,2560.000000,1.000000


In [57]:
final.to_csv('submissions/predictions_D_CUP_2.csv', index= None)